In [1]:
from jsonl2json import JsonlToJsonFormatter

jsonl = JsonlToJsonFormatter('pathways.jsonl', 'pathways.json')
jsonl.to_json()

In [2]:
import json

In [3]:
import re

In [4]:
import numpy as np

In [5]:
from bs4 import BeautifulSoup

In [6]:
import pandas as pd

In [7]:
with open("pathways.json") as f: 
    data = json.load(f)

In [8]:
dataf = pd.DataFrame(data)

In [9]:
# remove \n from col_2
dataf['name'] = dataf['name'].replace('\n', '', regex=True)# convert col_3 to list type

In [10]:
def get_phone_numbers(row):
    string = str(row.contact_html)
    soup = BeautifulSoup(string, 'html.parser')
    phone_pattern = "(\\d{3}[^a-zA-Z0-9_]\\d{3}[^a-zA-Z0-9_]\\d{4})"
    matches = re.findall(phone_pattern, string)
    if matches:
        matches = [ re.sub("([^a-zA-Z0-9_]|[-])", "", match) for match in matches ]
        return list(set(matches))
    else: 
        return None

In [11]:
def get_emails(row):
    string = str(row.contact_html)
    soup = BeautifulSoup(string, 'html.parser')
    links = soup.findAll('a')
    if links:
        emails = list()
        for link in links:
            url = link.get('href')
            match = re.search("mailto:", url)           
            if match:
                email = url.replace("mailto:", "")
                emails.append(email) 
        if emails:
            return list(set(emails))
        else:
            return None
    else:
        return None

In [12]:
def get_locations(row):
    string = str(row.contact_html)
    soup = BeautifulSoup(string, 'html.parser')
    links = soup.findAll('a')
    if links:
        locations = list()
        for link in links:
            url = link.get('href')
            match = re.search("https://maps.google.com", url)
            if match:
                locations.append(url)
        if locations:
            return list(set(locations))
        else:
            return None
    else:
        return None


In [13]:
def get_websites(row):
    string = str(row.contact_html)
    soup = BeautifulSoup(string, 'html.parser')
    links = soup.findAll('a')
    if links:
        urls = list()
        for link in links:
            url = link.get('href')
            match1 = re.search("https://maps.google.com", url)
            match2 = re.search("mailto:", url)
            if not match1 and not match2:
                if not re.match(r"^/", url):
                    urls.append(url)
        if urls:
            return list(set(urls))
        else:
            return None
    else:
        return None

In [14]:
dataf['phone_numbers'] = dataf.apply(lambda row: get_phone_numbers(row), axis=1)
dataf['emails'] = dataf.apply(lambda row: get_emails(row), axis=1)
dataf['location_maps'] = dataf.apply(lambda row: get_locations(row), axis=1)
dataf['websites'] = dataf.apply(lambda row: get_websites(row), axis=1)

In [15]:
data_all = dataf.drop(['contact_html'], axis=1)

In [16]:
data_all = data_all.explode('description')

In [17]:
data_all.fillna("", inplace=True)

In [18]:
data_all['category'] = data_all['category'].apply(', '.join)
data_all['phone_numbers'] = data_all['phone_numbers'].apply(', '.join)
data_all['emails'] = data_all['emails'].apply(', '.join)
data_all['location_maps'] = data_all['location_maps'].apply(', '.join)
data_all['websites'] = data_all['websites'].apply(', '.join)
data_all = data_all[['name', 'description', 'category', 'phone_numbers', 'emails', 'websites', 'location_maps', 'url']]

In [19]:
import gspread

gc = gspread.oauth()
sh = gc.open('MHRPSI System Navigation Knowledgebase')
worksheet = sh.worksheet("Pathways Resources")

In [65]:
pathways_topics = sorted(list(pd.melt(data_all.category.str.split(",", expand=True)).value.str.strip().dropna().unique()))
pathways_topics.remove('')

In [72]:
pathways_topics_df = pd.DataFrame(pathways_topics, columns=["Pathways Topic"])

In [73]:
pathways_topics_df

,Pathways Topic
0,Abuse / Neglect
1,Addictions / Substance Use
2,Advocacy
3,Cancer Care
4,Caregiver Support
5,Child Services
6,Condition Specific Support
7,Disability Services
8,Education
9,Employment


In [67]:
# export topic list to GSheets
pathways_topic_sheet = sh.add_worksheet(title="pathways_topics", rows=1000, cols=20)


In [74]:
pathways_topic_sheet.update([pathways_topics_df.columns.values.tolist()] + pathways_topics_df.values.tolist())


{'spreadsheetId': '1ekjejbzWvCucQk4t0xfJFJaqXXW0fGPEgz-LHGBfJg0',
 'updatedRange': 'pathways_topics!A1:A37',
 'updatedRows': 37,
 'updatedColumns': 1,
 'updatedCells': 37}